(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Parsing an xml file containing questions and its responses from Stack Overflow

##  Example: Stack Exchange API request to get all questions asked

Let's make a sample request to retrieve the questions posted on Stack Exchange on the first day of 2015. Documentation of the Stack Exchange API can be found [here](https://api.stackexchange.com/docs).

In [1]:
import requests

start_time = 1420070400 # 01-01-2015 at 00:00:00
end_time   = 1420156800 # 01-02-2015 at 00:00:00

response = requests.get("https://api.stackexchange.com/2.2/questions?pagesize=100" +
                        "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +
                        "&order=asc&sort=creation&site=stackoverflow")

print ("https://api.stackexchange.com/2.2/questions?pagesize=100" +
                        "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +
                        "&order=asc&sort=creation&site=stackoverflow")

https://api.stackexchange.com/2.2/questions?pagesize=100&fromdate=1420070400&todate=1420156800&order=asc&sort=creation&site=stackoverflow


All dates in the Stack Exchange API are in [unix epoch time](https://en.wikipedia.org/wiki/Unix_time). The format for the request string is specified [here](https://api.stackexchange.com/docs/questions).

We can try to print the response that Stack Exchange returns.

In [2]:
print response.text

It is easier to decode the raw response as JSON and use the `json` library to print it.

In [3]:
import json
print json.dumps(response.json(), indent=2)

{
  "has_more": true, 
  "items": [
    {
      "is_answered": true, 
      "view_count": 216, 
      "tags": [
        "php", 
        "mysql", 
        "arrays", 
        "mysqli"
      ], 
      "last_activity_date": 1420071815, 
      "answer_count": 2, 
      "creation_date": 1420070458, 
      "score": 0, 
      "link": "http://stackoverflow.com/questions/27727385/inserting-multiple-records-into-mysql-from-an-array", 
      "accepted_answer_id": 27727478, 
      "owner": {
        "user_id": 3210431, 
        "profile_image": "https://i.stack.imgur.com/QTjBV.jpg?s=128&g=1", 
        "user_type": "registered", 
        "reputation": 65, 
        "link": "http://stackoverflow.com/users/3210431/salep", 
        "accept_rate": 79, 
        "display_name": "salep"
      }, 
      "title": "Inserting multiple records into MySQL from an array", 
      "question_id": 27727385
    }, 
    {
      "is_answered": false, 
      "view_count": 397, 
      "tags": [
        "apache-pig"
      ]

Now we can easily see that the response consists of a list of question items. For each of these items, we get information about its attributes such as its `creation_date`, `answer_count`, `owner`, `title`, etc.

## Parsing an XML file containing all posts (questions and their responses) 

The following function parses out the questions posted in 2015. These are posts with `PostTypeId=1`. Finally we make a `pandas DataFrame` with 4 columns: `Id`, `CreationDate`, `OwnerUserId`, and the first tag in `Tags`. Save the `DataFrame` to a file named `question_dataframe.csv`.

In [4]:
import xml.etree.ElementTree as etree
import pandas as pd
def parseout(xmlfile):
    Id = []
    CreationDate = []
    OwnerUserId = []
    FirstTag = []
    for event, elem in etree.iterparse(xmlfile,events = ('start', 'end', 'start-ns', 'end-ns')):
        if event == "start":
            if elem.tag == "row":
                postTypeId = elem.get("PostTypeId")
                if postTypeId == "1":
                    Id.append(elem.get("Id"))
                    CreationDate.append(elem.get("CreationDate"))
                    OwnerUserId.append(elem.get("OwnerUserId"))
                    tags = elem.get("Tags")
                    if tags[0] =="<":
                        l = tags.index(">")
                        tags = tags[1:l]
                    FirstTag.append(tags)
                        
    data = {"Id": Id,
           "CreationDate" : CreationDate,
           "OwnerUserId" : OwnerUserId,
           "FirstTag" : FirstTag}
    
    frame = pd.DataFrame(data)
    frame.to_csv("question_dataframe.csv")
                
            
            
            

Below our code is used on a subset of the original file.

In [6]:
import pandas as pd
k = open("stackoverflow-posts-2015-subset.xml", "r")
parseout(k)

question_df = pd.read_csv("question_dataframe.csv")
print question_df

     Unnamed: 0             CreationDate               FirstTag        Id  \
0             0  2015-01-01T00:00:58.253                    php  27727385   
1             1  2015-01-01T00:01:43.673             apache-pig  27727388   
2             2  2015-01-01T00:02:32.123                    ios  27727391   
3             3  2015-01-01T00:02:57.983             sql-server  27727393   
4             4  2015-01-01T00:03:31.337                    php  27727394   
5             5  2015-01-01T00:04:01.407                android  27727396   
6             6  2015-01-01T00:05:03.773                     c#  27727406   
7             7  2015-01-01T00:05:27.167                   java  27727407   
8             8  2015-01-01T00:05:34.733                     c#  27727408   
9             9  2015-01-01T00:06:17.720                 apache  27727409   
10           10  2015-01-01T00:06:41.067                  mysql  27727410   
11           11  2015-01-01T00:07:28.747             javascript  27727414   

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()